In [3]:
#installing a libraries not found 
!pip install psycopg2-binary


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.8 MB/s eta 0:00:00a 0:00:01


In [4]:
#importing neccessarily libraries 
import psycopg2
import pandas as pd
import re 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from transformers import pipeline
#importing libraries to handle imabalance classes 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

2024-05-18 18:31:31.231104: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 18:31:31.231213: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 18:31:31.373948: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="Vetassist",
    user="niphemi.oyewole",
    password="W7bHIgaN1ejh",
    host="ep-delicate-river-a5cq94ee-pooler.us-east-2.aws.neon.tech",
    port="5432"
)


In [6]:
# Query to extract comments
query = "SELECT * FROM public.reddit_usernames_comments LIMIT 500"  # Limit to a small subset 
df = pd.read_sql_query(query, conn)


/tmp/ipykernel_34/3371540198.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [7]:

#check top 5 records in the dataset 
df.head()

,username,comments
0,LoveAGoodTwist,"Female, Kentucky. 4 years out. Work equine on..."
1,wahznooski,"As a woman of reproductive age, fuck Texas|As ..."
2,Churro_The_fish_Girl,what makes you want to become a vet?|what make...
3,abarthch,"I see of course there are changing variables, ..."
4,VoodooKing,I have 412+ and faced issues because wireguard...


In [8]:
# Preprocess comments (basic text cleaning)
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove user @ references and '#' from text
    text = re.sub(r'\@\w+|\#','', text)
    # Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lower case
    text = text.lower()
    return text

df['cleaned_comments'] = df['comments'].apply(lambda x: preprocess_text(x))

In [9]:
#check the cleaned comments 
df.head()

,username,comments,cleaned_comments
0,LoveAGoodTwist,"Female, Kentucky. 4 years out. Work equine on...",female kentucky 4 years out work equine only ...
1,wahznooski,"As a woman of reproductive age, fuck Texas|As ...",as a woman of reproductive age fuck texasas a ...
2,Churro_The_fish_Girl,what makes you want to become a vet?|what make...,what makes you want to become a vetwhat makes ...
3,abarthch,"I see of course there are changing variables, ...",i see of course there are changing variables b...
4,VoodooKing,I have 412+ and faced issues because wireguard...,i have 412 and faced issues because wireguard ...


In [10]:
# Initialize the LLM pipeline classifier model 
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define labels
candidate_labels = ["Medical Doctor", "Veterinarian", "Other"]

# Use the LLM to label the comments
def label_with_llm(comment):
    result = classifier(comment, candidate_labels)
    return result['labels'][0]

df['label'] = df['cleaned_comments'].apply(lambda x: label_with_llm(x))

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
#check the labels 
df.head()

,username,comments,cleaned_comments,label
0,LoveAGoodTwist,"Female, Kentucky. 4 years out. Work equine on...",female kentucky 4 years out work equine only ...,Other
1,wahznooski,"As a woman of reproductive age, fuck Texas|As ...",as a woman of reproductive age fuck texasas a ...,Other
2,Churro_The_fish_Girl,what makes you want to become a vet?|what make...,what makes you want to become a vetwhat makes ...,Veterinarian
3,abarthch,"I see of course there are changing variables, ...",i see of course there are changing variables b...,Other
4,VoodooKing,I have 412+ and faced issues because wireguard...,i have 412 and faced issues because wireguard ...,Other


In [12]:
#check the target values showing imbalance 
df['label'].value_counts()

label
Other             452
Veterinarian       41
Medical Doctor      7
Name: count, dtype: int64

In [13]:
#dropped unneccessarily columns 
df.drop(['username', 'comments'],axis=1, inplace=True)

TF-IDF Vectorization: Text data needs to be converted into a numerical format that can be used by machine learning algorithms. TF-IDF (Term Frequency-Inverse Document Frequency) is a common method for transforming text into vectors of numbers, reflecting the importance of words in the documents relative to the entire dataset. This approach helps to downscale the impact of frequently occurring words that are less informative (e.g., "the", "and") and emphasize rarer, more informative terms.

In [51]:
# Encode labels
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['label'])

# Split the data into features and target
X = df['cleaned_comments']
y = df['encoded_label']

# Create a TF-IDF vectorizer
tfidf = TfidfVectorizer()

# Fit and transform the text data to numerical format
X_tfidf = tfidf.fit_transform(X)

# Apply SMOTE to balance the classes
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

# Split the resampled data into training and evaluation sets
X_train, X_eval, y_train, y_eval = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Create a pipeline for different models
pipelines = {
    'svm': Pipeline([('svm', SVC(probability=True))]),
    'random_forest': Pipeline([('random_forest', RandomForestClassifier())]),
}

# Define hyperparameters for grid search
param_grids = {
    'svm': {
        'svm__kernel': ['linear', 'rbf'],
        'svm__C': [0.1, 1, 10]
    },
    'random_forest': {
        'random_forest__n_estimators': [50, 100, 200],
        'random_forest__max_depth': [None, 10, 20]
    }
}

# Perform grid search with cross-validation for each model
best_estimators = {}
for model_name, pipeline in pipelines.items():
    grid_search = GridSearchCV(pipeline, param_grids[model_name], cv=5, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_estimators[model_name] = grid_search.best_estimator_
    print(f'Best parameters for {model_name}: {grid_search.best_params_}')
    y_pred = grid_search.predict(X_eval)
    accuracy = metrics.accuracy_score(y_eval, y_pred)
    print(f'Accuracy for {model_name}: {accuracy}')
    print(metrics.classification_report(y_eval, y_pred, target_names=label_encoder.classes_))

Best parameters for svm: {'svm__C': 10, 'svm__kernel': 'rbf'}
Accuracy for svm: 1.0
                precision    recall  f1-score   support

Medical Doctor       1.00      1.00      1.00       104
         Other       1.00      1.00      1.00        83
  Veterinarian       1.00      1.00      1.00        85

      accuracy                           1.00       272
     macro avg       1.00      1.00      1.00       272
  weighted avg       1.00      1.00      1.00       272

Best parameters for random_forest: {'random_forest__max_depth': None, 'random_forest__n_estimators': 200}
Accuracy for random_forest: 0.9448529411764706
                precision    recall  f1-score   support

Medical Doctor       1.00      0.99      1.00       104
         Other       0.86      0.98      0.92        83
  Veterinarian       0.97      0.86      0.91        85

      accuracy                           0.94       272
     macro avg       0.95      0.94      0.94       272
  weighted avg       0.95     

In [56]:
# Choose the best model based on accuracy or other metrics
best_model = best_estimators['svm']

In [57]:
# Test the model with the provided text
test_text = """
I think a lot of other people made some good points so Iâ€™m not going to repeat a lot of it. Itâ€™s never too late to consider vet med as a career and undergrad is a great opportunity to push yourself and make yourself competitive.

I will play devils advocate though. I am a second year resident and Iâ€™m burnt out already. Undergrad is 4 years, vet school is another 4, and residency is 3. If you want to do aquatic medicine then it will likely be another 2 years of rotating and specialty internships before residency. Thatâ€™s upwards of 13 years of training ahead of you. Vet Med can be rewarding, but it is hard and long to get to the end of your training. Iâ€™m not saying this to discourage you, but simply to make sure you know what you would be commuting to. You have to be driven and always keep your eye on the prize. Be prepared for long nights, long weeks, and long years.|As someone who knows people looking for path jobs in diagnostics/industry, this information is not current. Most diagnostic centers have placed a hold on hiring and industry has definitely slowed down. Also, pharma almost always requires a concurrent PhD.|Some of these comments are absolutely insane and just speak to how toxic the culture is in vet med. For most of you, stop cherry picking what OP said in the post. If the technician is calling the other doctors by their last name/title and refusing to call OP by it then itâ€™s not a lack of respect, it is active disrespect. As a technician, I would never have disrespected a doctor like that. As a doctor now, I would expect the same respect as my other colleagues. I go by my first name because thatâ€™s how I like it, but as someone who was once a technician, the appropriate thing to do is to call them by their last name/title at first and feel it out or simply ask the what they prefer.

For those of you saying that respect is earned, OP earned that shit by completing 8+ years of schooling to get their degree (13+ for some of us). Did they know everything coming out? No. None of us do. Most of us understand that and are welcome to constructive advice (which I think itâ€™s clear that OP is). What should not be acceptable is the nitpicking and hazing that is often given to new grads. There is a way to respectfully give advice. If you are a technician OR a doctor, this is a useful skill to learn. Technicians know things doctors donâ€™t know and doctors donâ€™t know things technicians know. Instead of belittling each other, they should create an environment where people are open to ask questions and share advice.|I think a lot of other people made some good points so Iâ€™m not going to repeat a lot of it. Itâ€™s never too late to consider vet med as a career and undergrad is a great opportunity to push yourself and make yourself competitive.

I will play devils advocate though. I am a second year resident and Iâ€™m burnt out already. Undergrad is 4 years, vet school is another 4, and residency is 3. If you want to do aquatic medicine then it will likely be another 2 years of rotating and specialty internships before residency. Thatâ€™s upwards of 13 years of training ahead of you. Vet Med can be rewarding, but it is hard and long to get to the end of your training. Iâ€™m not saying this to discourage you, but simply to make sure you know what you would be commuting to. You have to be driven and always keep your eye on the prize. Be prepared for long nights, long weeks, and long years.
"""

predicted_label = predict_label(test_text)
print(f'Predicted label: {predicted_label}')

Predicted label: Veterinarian


In [58]:
# Test the model with the provided text
test_text = """
"If you like the industry but donâ€™t care for the pay an employee in a clinic makes you can always look at industry roles. Idexx,zoetis,Heska,petdesk, there are thousands of veterinary companies out there that offer better pay and allow you to still be in the field. Although not in practiceâ€¦|There is a lower the dose campaign from idexx. You can find it on the website I am sure. They also sell the only digital radiology system that requires half the radiation dose than that of a traditional system. Pretty cool technology, maybe your clinic is willing to check it out.|Yâ€™all wild. Iâ€™ll let yâ€™all continue to swirl around this toilet bowl of a discussion. Sorry to derail your question op. The pile of shit being left here isnâ€™t getting you anywhere closer to finding a worthwhile associate/partner. And to the clown that deleted their postsâ€¦.foreshame. Internet is full of unworthy trolls and youâ€™ve proven one.|Bless all your hearts. This isnâ€™t a difficult game to figure out. Yâ€™all seem to be too dense to look for a viable solution that involves compromises. Only dvms I know that are unhappy are corporate dvms that have to abide by the corp bullshit that anyone has a dislike for. You ainâ€™t happy where you at then fix it. Quit complying. 

This poor clinic just askin about how to find a vet now has to weed thru all this bullshit for no real answer other than find a non lgbtqia+-= male dvm. Otherwise it seems like a headache.|It is disappointing  to feel like the work ethic of this industry has declined over the last 20 years. 15 years ago new grads could expect base $55k with about 6/7% production. Now they are getting $100k 2/4% production with much better benefits and work life balance. Not to mention the sign on bonus that corporate groups are giving the new grads. All tides turn in time. I think youâ€™ve got a hell of an offer regardless of what the states reproductive rights are.|TouchÃ©|When the commenters here merely state TEXAS!?!? Fuck texas and their anti feminist anti gay bullshit? Clearly thatâ€™s someone watching too much news and not living day to day here. Op didnâ€™t even mention extended benefits and everyone slammed it as itâ€™s a texas problem. Itâ€™s not a texas problem. Obviously itâ€™s a workforce problem when the women are too scared or stubborn or both to come to a place that will pay them fairly. As op stated a pretty fair wage as far as Iâ€™m considered and still not knowing exact details.|Never once did I say pull anything up by any bootstraps. Yâ€™all putting words in my mouth. And vary rarely is an abortion life threatening. To wait a week or less to travel to an area that allows it isnâ€™t difficult. Please stay wherever you are and advocate whatever you want. This tribalism of red vs blue is only compounding this and many other issues.|If you canâ€™t abort an unwanted pregnancy shouldnâ€™t you be taking the act of unprotected sex into consideration as well. Donâ€™t want a baby? Thereâ€™s ways to prevent it to begin withâ€¦.|You can always travel for repro rights. Not a difficult concept. And most Texans donâ€™t give a shit what you identify as as long as you donâ€™t push your shit on us who cares what you do. Turn the tv off and have some conversations with people. The difference isnâ€™t a wide gap and we tend to agree on most things. Yâ€™all too worried about what msnbc or fox as to say about it. Their are not our puppet, or yours. Quit acting like it. And yeah I do believe the system can change. Iâ€™ve seen it happen. Iâ€™ve lived the change. To discount a place because of international food? You have clearly never experienced the vast culinary melting pot of many texas cities. Too stuck in your own existence to experience what different places have to offer. Keep yourself where you are itâ€™s your mindset that keeps shit stagnant.|I question your intelligence in the face of your advanced degree. Learn the industry and you may surprise yourself. Fucking clowns.|Supply and demand affect these rates the most. Start hurting for vets the wages go up. Also the type of medicine they practice matter. Small animal vs large animal, ecg.|Soooo if 15 years ago the going rate for a new grad was $55k and now, most places I deal with are offering $100kâ€¦â€¦ itâ€™s not quite double but pretty darn close. Seems like pay is keeping up with most things. Again you donâ€™t like your current situation, there are others to be had. There is just a lack of doing so for whatever reason you want to point out. I most commonly see what I see.|To say politics is the reason they canâ€™t hire anyone is flat bullshit that only the weak and simple minded allow them to let it affect them like that.|I would question her place of employment instead of the state she lived in. Thatâ€™s dumb.|I wouldnâ€™t require someone to do something with their body they didnâ€™t want to. Itâ€™s not my decision. And if the govt says itâ€™s a must, well time to adapt right? I personally wouldnâ€™t fire someone because they chose to or not to get something thatâ€™s voluntary. I helped my people look for alternatives to vaccination. Medical/religious exemptions. Flat saying no thanks no poke for me. I let my people choose. Honestly great point! Maybe we will implement some sort of compensation for women in my org that need to travel to get the medical care they require. I could even implement it for the opposite of women that need help getting pregnant. Looky here something positive from this shit storm.|Lol obviously not. But if I found myself in that situation Iâ€™ve got bigger fish to fry than an unwanted pregnancy. If thatâ€™s your true worry maybe you should reassess your surroundingsâ€¦possibly a state that allows open carry of a firearm might deter any unsavory types that could lead to said unwanted pregnancies?|Okurrr. Just look at movement maps during Covid. One of the most blue states, California, had more residents move from there to texasâ€¦ Jesus yâ€™all need to pop another ssri and chill. How do you think those drugs yâ€™all are on for such a hard life yâ€™all have had affect a fetus?|Never said I wanted them to that it was justified that they made that. Itâ€™s much different now especially in my area of texas, the average salary is $100k plus production for even a new grad.|My reason and logic seem to confuse even the most basic of individuals. My intelligence should a always be questioned. As should yours. If we are not flexing the gray matter on a daily basis we are but withering away.|Thatâ€™s ignorant to leave the industry because your not getting paid what you are worth. Find a place that will pay. They are everywhere. Of the 1000 plus vet clinics in my part of the world 80% are looking for a dvm and will pay $100K + incentives. Open your eyes. Either that or these drs leaving want to work 10-4 with a 2 hour lunch and 2 months pto. Oh and only want vaccine appointments because they arenâ€™t comfortable in sx or more difficult cases. Thatâ€™s the lack of gumption I speak of.|No one said to take out student loans. I know dvms that floated their schooling and I know ones that did take loans out and paid it off in 3 years because they busted ass. The way some of these commenters are talking they be living at the clinic so what mortgage? (Joking donâ€™t downvote me more). Sure things are more expensive I get that, everyone feels that bite. But if your wages in the profession are up you should still just as easily meet those obligations just as the dvms of 15 years ago did. Iâ€™m no math expert but I can do that math.|As the saying goes scared money donâ€™t make money. Itâ€™s like saying I wonâ€™t work in Houston because of greenspoint. Well why not Conroe instead? Might as well not go to New York either since we ship both the immigrants and our stray dog population there. Oh no, the migrant flow continuesssssss|Itâ€™s only hostile to them because they let it be. Ok my state says I can or canâ€™t do something, doesnâ€™t mean I have to abide by that. I bet you never speed do you? Thatâ€™s the state telling you what you can and canâ€™t do with your body too isnâ€™t it? A bit oversimplified but the same concept. Yâ€™all too worried about red vs blue or they vs them. You do you boo but please stay the fuck away from this state and itâ€™s wonderful people. Thereâ€™s a saying for a reason, donâ€™t California my Texas. The anger and downvotes Iâ€™ve gotten in this thread is hilarious. All the while I just laugh continue about my day. I bet yâ€™all going home and crying to your spouse about the unhinged asshole on Reddit. Cry into your pillow, no one cares.|Idk if you read my last comment. I literally just said I would like to look into compensation for women that require or want such care. Itâ€™s not their fault texas has this issue and they shouldnâ€™t suffer, no one should, because of it. Iâ€™m not denying anything. If anything Iâ€™m looking to facilitate their choiceâ€¦even would like to go as far as helping the women whom want to become pregnant that canâ€™tâ€¦ I fail to see your issue here.|Please read above. I stated I helped my people find alternatives. Religious and medical exemptionsâ€¦ I fail to see the issueâ€¦|Daily. I welcome it. If I donâ€™t challenge my customers and they donâ€™t challenge me itâ€™s a boring life. Hell, if I donâ€™t challenge myself how boring. Iâ€™ll be the first to admit when Iâ€™m wrong as well. Hence how Iâ€™ve made it to where I am in life and in this industry. Just because I donâ€™t have some advanced degree from some prestigious institution doesnâ€™t mean I lack in intelligence. Iâ€™ve given my life to this industry just as many veterinarians have. Iâ€™m not so pretentious that I canâ€™t see the Forrest for the trees. This answer is simple. Your undervalued and overworked? GO SOMEWHERE ELSE! The clinic you work at isnâ€™t the only one, itâ€™s not your only option. Not in Texas, not in Idaho. No matter where you are or want to be there are many options for yourself. You just have to take the initiative, rely on yourself, no one else.|If you canâ€™t afford to live and work where you currently are, and feel undervalued I would imagine doubling your base salary and increasing your other benefits would entice you to make the move across the country. No high horse just the reality of where I am at. Iâ€™m not happy where Iâ€™m at or getting paid what Iâ€™m worth Iâ€™m actively looking for a place I will be satisfied. Simple as that. And maybe they would off themselves if they realized that was an opportunity.|16 I know paid it off within 3 years. All of them in the last 8 years. How did they pay it off within 3 years of graduation you ask? By busting their ass and living below their means. Taking extra shifts at other hospitals. Hell one of them worked in industry, worked overnight ec and lived like they were in college still. It can happen. It does happen. Funny thing is we can circle back to work ethic and gumption. Those that choose to pay their loans off over the long haul, typically lack some of those two qualities. Or extenuating circumstances create issues paying off loans that fast. The power of simple math fired the ass busters up and they made it happen for themselves.|Yes customers. I spent 10 years in practice. In every role. As a industry consultant I speak with customer, vet clinics, all over the country. I advise them to stay away from and to not hire clown acts like yourself. Just because I currently reside here doesnâ€™t mean my view point would be different if I lived somewhere else. Like when I lived in California for 6 years. Still a consultant in the industry I directed my customer, clients, what have you on how to best run their business. Staying away from problem or toxic employees is at the top of the list. If you are a veterinarian I feel terrible for your clients and your business. Maybe itâ€™s not suffering now but a room full of toxic bullshit always boils over.|I havenâ€™t screamed once. And considering Iâ€™ve had to weather all types of economic storms Iâ€™m pretty well versed on it. Also, thanks for assuming the adversities I have faced in life. But if you must know my back story hereâ€™s a snippet. Grew up poor, busted my ass to put myself thru school and worked tirelessly for an industry I love and have literally almost died for. Yet Iâ€™m still here fighting the good fight. Politics be damned.|I would check diagnostic and distribution companies. Thereâ€™s all types of remote dvm roles. Something like idexx, telehealth or corporate veterinary groups. Not sure if any of those are in Germany but itâ€™s worth a shot."
"""

predicted_label = predict_label(test_text)
print(f'Predicted label: {predicted_label}')

Predicted label: Veterinarian


In [59]:
# Test the model with the provided text
test_text = """
"In phoenix, we are offering our new grads prosal 135k with  19% and a 40k signing bonus. With no negative accruals.  Just food for thought.|You can always try. Worst they say is no.|I would focus on dog, cat, horse and bovine questions. Have a high percentage of correct answers in these for vetprep. (Maybe +/- swine.) Know your weaknesses- example - if itâ€™s pharm really study MOAs, etc. Know common diseases, testing modalities, treatments for these species.   


Dont stress about the other species- they make up such a small percentage of the exam that they should not be stressed about and an educated guess is good enough. 

Also agree using the ICVA practice test. Itâ€™s formatted like the real thing and gets you a good idea about passing/score wise.|In phoenix, we are offering our new grads prosal 135k with  19% and a 40k signing bonus. With no negative accruals.  Just food for thought.|You can always try. Worst they say is no.|I would focus on dog, cat, horse and bovine questions. Have a high percentage of correct answers in these for vetprep. (Maybe +/- swine.) Know your weaknesses- example - if itâ€™s pharm really study MOAs, etc. Know common diseases, testing modalities, treatments for these species.   


Dont stress about the other species- they make up such a small percentage of the exam that they should not be stressed about and an educated guess is good enough. 

Also agree using the ICVA practice test. Itâ€™s formatted like the real thing and gets you a good idea about passing/score wise.|In phoenix, we are offering our new grads prosal 135k with  19% and a 40k signing bonus. With no negative accruals.  Just food for thought.|You can always try. Worst they say is no.|I would focus on dog, cat, horse and bovine questions. Have a high percentage of correct answers in these for vetprep. (Maybe +/- swine.) Know your weaknesses- example - if itâ€™s pharm really study MOAs, etc. Know common diseases, testing modalities, treatments for these species.   


Dont stress about the other species- they make up such a small percentage of the exam that they should not be stressed about and an educated guess is good enough. 

Also agree using the ICVA practice test. Itâ€™s formatted like the real thing and gets you a good idea about passing/score wise.|In phoenix, we are offering our new grads prosal 135k with  19% and a 40k signing bonus. With no negative accruals.  Just food for thought.|You can always try. Worst they say is no.|I would focus on dog, cat, horse and bovine questions. Have a high percentage of correct answers in these for vetprep. (Maybe +/- swine.) Know your weaknesses- example - if itâ€™s pharm really study MOAs, etc. Know common diseases, testing modalities, treatments for these species.   


Dont stress about the other species- they make up such a small percentage of the exam that they should not be stressed about and an educated guess is good enough. 

Also agree using the ICVA practice test. Itâ€™s formatted like the real thing and gets you a good idea about passing/score wise.|In phoenix, we are offering our new grads prosal 135k with  19% and a 40k signing bonus. With no negative accruals.  Just food for thought.|You can always try. Worst they say is no.|I would focus on dog, cat, horse and bovine questions. Have a high percentage of correct answers in these for vetprep. (Maybe +/- swine.) Know your weaknesses- example - if itâ€™s pharm really study MOAs, etc. Know common diseases, testing modalities, treatments for these species.   


Dont stress about the other species- they make up such a small percentage of the exam that they should not be stressed about and an educated guess is good enough. 

Also agree using the ICVA practice test. Itâ€™s formatted like the real thing and gets you a good idea about passing/score wise.|In phoenix, we are offering our new grads prosal 135k with  19% and a 40k signing bonus. With no negative accruals.  Just food for thought.|You can always try. Worst they say is no.|I would focus on dog, cat, horse and bovine questions. Have a high percentage of correct answers in these for vetprep. (Maybe +/- swine.) Know your weaknesses- example - if itâ€™s pharm really study MOAs, etc. Know common diseases, testing modalities, treatments for these species.   


Dont stress about the other species- they make up such a small percentage of the exam that they should not be stressed about and an educated guess is good enough. 

Also agree using the ICVA practice test. Itâ€™s formatted like the real thing and gets you a good idea about passing/score wise."
"""

predicted_label = predict_label(test_text)
print(f'Predicted label: {predicted_label}')

 #C:\Users\USER\.dbt\profiles.yml

Predicted label: Veterinarian


In [ ]:
43d987d8d05b

#docker cp "C:\Users\USER\Downloads\datatraining\jaffle-data" 43d987d8d05b:/tmp/airbyte_local